In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

from datetime import date
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

In [3]:
# Tickers
MAG7_TICKERS = ['AAPL','MSFT','TSLA','NVDA','META','AMZN','GOOG']
MAG7_WEIGHTS = [0.0656,0.0643,0.0225,0.0699,0.0264,0.0426,0.0225]

INDEX_TICKER = ['SPY']
EXTRA_TICKERS = ['AES', 'LNT', 'AEE', 'AEP', 'AWK', 'ATO', 'CNP', 'CMS', 'ED']

# Start/End Dates
start_date = date(2010,1,1)
end_date = date(2025,1,1)

# Yfinance Download
df_index = yf.download(tickers = INDEX_TICKER,start = start_date, end = end_date, auto_adjust = True)['Close']
df_mag7 = yf.download(tickers = MAG7_TICKERS,start = start_date, end = end_date, auto_adjust = True)['Close']
df_extra = yf.download(tickers = EXTRA_TICKERS,start = start_date, end = end_date, auto_adjust = True)['Close']

# Log Returns
df_index_ret = (df_index
             .pct_change()
             .dropna()
        )

df_mag7_ret = (df_mag7
                .pct_change()
                .dropna()
        )

df_extra_ret = (df_extra
             .pct_change()
             .dropna()
        )

df_mag7_ret.index = pd.to_datetime(df_mag7_ret.index).date

# Weighted MAG7 Returns (Estimate)
# df_mag7_ret['Seven_Weighted'] = np.sum(df_mag7_ret*MAG7_WEIGHTS, axis = 1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  9 of 9 completed


In [4]:
# Imports
tickers = ["AAPL", "NVDA", "META", "GOOG", "AMZN", "MSFT", "TSLA"]  
dataframes = {ticker: pd.read_csv(f"../../data/{ticker}.O.csv", index_col=0) for ticker in tickers}  
spx = pd.read_csv('../../data/SPX.GI.csv',index_col=0)
# Concatenate all dataframes
df = pd.concat(dataframes, names=["Ticker"])  

# Format df
df = (df
      .reset_index()
      .rename(columns = {"level_1":'date','MKT_CAP_ARD':'market_cap'})
      .pivot(index = 'date',columns='Ticker',values = 'market_cap')
)

# Divide to get time series weights

mag7_weights = df.div(spx['MKT_CAP_ARD'],axis = 0).dropna().sum(axis = 1)
# df = df.div(spx['MKT_CAP_ARD'],axis = 0).dropna().sum(axis = 1)
df = df.div(df.sum(axis = 1,skipna=True),axis = 0)
df = df.fillna(0)
df.index = pd.to_datetime(df.index).date

In [5]:
common_index = df.index.intersection(df_mag7_ret.index)

# Align both DF
df_weights = df.loc[common_index]
df_index_ret = df_index_ret.loc[common_index]
df_mag7_ret_aligned = df_mag7_ret.loc[common_index]

In [6]:
def rolling_beta(portfolio_returns, market_returns, window):
    betas = []
    for i in range(window, len(portfolio_returns)):
        X = market_returns[i - window:i].values.reshape(-1,1)
        y = portfolio_returns[i - window:i]
        model = LinearRegression().fit(X, y)
        betas.append(model.coef_[0])
    return pd.Series(betas, index=portfolio_returns.index[window:])

In [7]:
beta_series = {}
for ticker in df_mag7_ret_aligned.columns:
    beta_series[ticker] = rolling_beta(df_mag7_ret_aligned[ticker],df_index_ret['SPY'],252)

beta_ts = pd.DataFrame(beta_series).dropna()

# Needs to be fixed
weighted_beta = (beta_ts*df_weights).sum(axis = 1)
weighted_beta = weighted_beta[weighted_beta > 0]

In [10]:
mag7_weights.index = pd.to_datetime(mag7_weights.index).date 
weighted_beta.index = pd.to_datetime(weighted_beta.index).date 

common_index = weighted_beta.index.intersection(mag7_weights.index)

# Align both DF
mag7_weights =  mag7_weights.loc[common_index]
weighted_beta =  weighted_beta.loc[common_index]

In [11]:
weighted_beta = (weighted_beta*mag7_weights)

*Plot*

In [313]:
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = weighted_beta.index,
        y = weighted_beta,
        name = 'Magnificent Seven',
        mode='lines',
        line=dict(width=0.5, color='rgb(131, 90, 241)'),
        stackgroup='one',
        groupnorm='percent'
    )
)

fig.add_trace(go.Scatter(
    x= weighted_beta.index,
    y = 1 - weighted_beta, 
    mode='lines',
    name = 'Rest of Index',
    line=dict(width=0.5, color='rgb(127, 166, 238)'),
    stackgroup='one'
))

fig.update_layout(title = 'Weighted Index Beta Contribution',
                  showlegend=True,yaxis=dict(type='linear',range=[1, 100],ticksuffix='%'),
                  margin=dict(l=10, r=10, t=50, b=10),
                  legend=dict(orientation="h",yanchor="top",y=-0.1,xanchor="center",x=0.5)
                )
fig.update_yaxes(title = "Contribution %")
fig.update_xaxes(title = 'Date')
fig.show()

In [23]:
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = mag7_weights.index,
        y = mag7_weights*100,
        name = 'Mag 7 Weight',
        mode='lines',
    )
)


fig.update_layout(title = 'Magnificent 7 Index Weight',
                  showlegend=True,yaxis=dict(type='linear',range=[1, 100],ticksuffix='%'),
                  margin=dict(l=10, r=10, t=50, b=10),
                  legend=dict(orientation="h",yanchor="top",y=-0.1,xanchor="center",x=0.5),
                  xaxis_title = 'Date',
                  xaxis=dict(title_standoff=3) 
                )
fig.update_yaxes(title = "Weight (%)")
fig.show()